<a href="https://colab.research.google.com/github/QaziSaim/CASE-STUDIES/blob/main/EasyOCRISGood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers qwen_vl_utils accelerate>=0.26.0 PEFT -U
# !pip install -U bitsandbytes


In [2]:
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 9.4 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os
os.environ['HUGGINGFACE_API_KEY']=userdata.get('HUGGINGFACE_API')

In [9]:
path = '/content/image_two.png'

In [10]:
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import torch
import os
from qwen_vl_utils import process_vision_info

# ---- 1️⃣ Define your image path ----
image_path = path  # <<=== put your image filename here

# ---- 2️⃣ Load model and processor ----
model_name = "oddadmix/Qaari-0.1-Urdu-OCR-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_name)

max_tokens = 2000
prompt = (
    "Below is the image of one page of a document. "
    "Return the plain Urdu text as if you were reading it naturally. "
    "Do not hallucinate."
)

# ---- 3️⃣ Load and (optionally) re-save image ----
image = Image.open(image_path).convert("RGB")
image.save("image_temp.png")  # optional, if you want a local temp copy

# ---- 4️⃣ Prepare chat message for vision input ----
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{os.path.abspath(image_path)}"},
            {"type": "text", "text": prompt},
        ],
    }
]

# ---- 5️⃣ Process and run inference ----
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=max_tokens)
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)[0]

# ---- 6️⃣ Show result ----
print("📝 Extracted Urdu Text:\n")
print(output_text)

# (Optional) Clean up temp file
os.remove("image_temp.png")


📝 Extracted Urdu Text:

ماڈل تاریق نے فراہمیاں میں اندازاتوں کے ساتھ گزشتہ جنگاتے کے والکے کفرہوں نے اورلوں اں کٹوئل کرلیں تو سب کافرہا ہے جے کے ساتھ کرلے جنگاتے کے ساتھ محدودیتیں اور کہیں شیاں تاہم دیاوا کیا اوراکھا کیا کے رواجات کے کلے کفرہوں نے اورلوں نے واقعات کے ساتھ گھنگھا چپے رہے اوراں کے ساتھ کفرہوں نے اورلوں نے کام کے اعتراف کرے اورا کرکٹر کے 2021ءاں طریقے کا دو چوڑگی کی گئیںگھنگھے کے ساتھ کی کمپنیاں وابجے سے اوران کی عزت ودھان کے حیات سخت نے اوران کی فخرت نے اے ایڈ اوگنگ کے حیات طاہری میں اور اقتیال فراہے نے اعراجان کا کہے گئے ہمیشہ پرکم پر فزگے تہے جے گئے جب اس کی کہہکاشت ودھٹ کے ساتھ کے ذریعے تاحیے ہوں اوراں کے ذریعے محقق کاصل ہوجے رہے اوراں کے شفیل کے کام کرپرہوں پاپلی گوئل کے ذریعے فزگے ساکت گا چپے گے اورا کے چاہرے ہائی گے گے گاہول کی نہایت کے لیے اوراکھا ہاتے گے چزے پی چتھے کے لیے اوراکھا کا ہاتے گے چزے پی چتھے کے لیے اوراکھا کا ہاتے گے چزے پی چتھے کے لیے اوراکھا کا ہاتے گے چزے پی چتھے کے لیے اوراکھا کا ہاتے گے چزے پی چتھے کے لیے اوراکھا کا ہاتے گے چزے پی چتھے کے لیے اور

In [11]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 30.6 MB/s eta 0:00:00


In [12]:
import easyocr

reader = easyocr.Reader(['ur'])  # Urdu
results = reader.readtext(path)

for (_, text, conf) in results:
    print(f"{text} (Confidence: {conf:.2f})")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Completeطای قارینفرایا" جمعالفتاوی 'مسےآکروعظ وھیحت کنےوالار : (Confidence: 0.07)
کفربولے اورلوگ اسل کوقبول کرلیں توسب کافر اوجامی ے ءالۓے كلوک (Confidence: 0.04)
جہلے ببمعذوریں اور" میط'ممی اتتازیادوکيا : اورکها کیا کماکرداعظے (Confidence: 0.05)
کر کفربوۓے بعدلوگول_ واعظے کچھنہکہاچپپ رےءاورایے (Confidence: 0.12)
البےرے کافرتاوجامں _ ولیمن جکهییجات ہول کدوا ییبات کرەكر (Confidence: 0.01)
کافرہوکیا ءیااسی کكلام پراعقادکری اوراگرقاآل ]٢ے'[ای ریتے کا (Confidence: 0.06)
وە چرجی نیق ےحتق ی نہبانی واجب ےءاوران کیعزت دنامول ک (Confidence: 0.03)
حمایت تعین ےء اوران یانحرت ءانےایذا ودفع کرےےحیات ناریی (Confidence: 0.08)
اوراغقال فرانے بعدان کاحقےین برمون پفضینےءین جب (Confidence: 0.09)
اسق یانگہداشت ومخص کرےجسس ے ذرلیوق غالب او٠ اُورای ۓ (Confidence: 0.02)
قم ظابرہواوباق لوکول (Confidence: 0.13)
ذرلیوای مقدمہ کافیصل 8وجاۓ ُاورال کنیا (Confidence: 0.03)
کذممےفض ساقط اوجاۓ کاءاوراخباب کاحم باق رےکاء واہول ك (Confidence: 0.09)
ز

In [15]:
!sudo apt install tesseract-ocr-urd
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-urd is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [16]:
import pytesseract
from PIL import Image

img = Image.open(path)
text = pytesseract.image_to_string(img, lang='urd')
print(text)


طائی ا رکی نے فر مایا مع اتا وی یس ہے اگر وعنا یح ت ککینے دا لاک
کفربو لے اور لوگ ا سکوقو لک ریش تے س بکافر ہو ہیں گے اسل ےکراوگ
لی کےس بب معنر ور او مس اتا ز یاد کیا :او کہاگ یا گر داع کے
کل فکف ہو لے کے بعداوکوں نے داخظ سے بن ہکہا چپ رہے :اددال کے
پاکی ٹیٹھ دہ ہکافر ہو ای کے مین خی جات ہو ںکردوی با تک ےر
کاخ ہوگیاءیاال کےککام پرا تق کی او راگ ال (۴ے٣)‏ اس مر یق کا نہ
بہ گرم ن لنٹ ےت کی نکہبانی واجپ ےءاوران: عزت دنا مو لک
ات ضتان ہے+اورا نکی ضصرت ءان سے انذ اود کر کے ضیات نا ہرکی ٹل
اوہ اتال فرمانے کے بحدا نکاتقی ہے ہش ہ ون پرفر مین ہے یکن جب
ا ق نکی گہداشت وچنخ سکرے جس کے ذد کرت غااب ہوہ اود ای کے
ذدر یراس مقد کا فیصلہہوجاۓ :اوراس کے ٹیم ےگ ظاہر ہاو باق یں
کے ذم سے فرص ساقط ہو جا ۓےگاء اد ابا بکاعم باقی ر ہےگاءگواہو لک
زیادٹی کے لع اوداری بات سےتھ وک یتقو یت کے لے ءاورسل فکااس بات
باہمارئ ےکعدث ٹس متھ کا حال ہما نکیا جات ےگا شڑنی ال لک ردیت ال

